In [26]:
from datasets import load_dataset

dataset = load_dataset('emotion')

label_mapping = {
    0: "sadness",
    1: "joy",
    2: "love",
    3: "anger",
    4: "fear",
    5: "surprise",
}
emotions = ["sadness", "joy","love","anger","fear", "surprise"]
test_dataset = dataset['validation']

In [42]:
import ollama

def get_llm_response(prompt):
    response = ollama.generate(model='mistral', prompt=prompt)
    return response['response'].strip()

def classify_emotion(text):
    prompt = f"Classify the emotion in the following sentence with only one word. The emotion has to be one of the following: {', '.join(emotions)}. Don't explain your choice.\nSentence: '{text}'\nEmotion:"
    return get_llm_response(prompt)

In [43]:
import tqdm

correct = 0
total = len(test_dataset)

sample_responses = []

for example in tqdm.tqdm(test_dataset):

    text = example['text']
    expected_label = label_mapping[example['label']]
 
    predicted_label = classify_emotion(text)
    

    if expected_label.lower() in predicted_label.lower() :
        correct += 1
    
    sample_responses.append({
        "text": text,
        "predicted_label": predicted_label,
        "expected_label": expected_label,
    })

accuracy = correct / total * 100

100%|██████████| 2000/2000 [04:28<00:00,  7.44it/s]


In [44]:

print(f"Accuracy modelu: {accuracy:.2f}%")
print("\nPrzykładowe odpowiedzi modelu:")
for response in sample_responses[:5]:
    print(f"Tekst: {response['text']}")
    print(f"Odpowiedź modelu: {response['predicted_label']}")
    print(f"Oczekiwane: {response['expected_label']}")

Accuracy modelu: 40.35%

Przykładowe odpowiedzi modelu:
Tekst: im feeling quite sad and sorry for myself but ill snap out of it soon
Odpowiedź modelu: Sadness
Oczekiwane: sadness
Tekst: i feel like i am still looking at a blank canvas blank pieces of paper
Odpowiedź modelu: Sadness
Oczekiwane: sadness
Tekst: i feel like a faithful servant
Odpowiedź modelu: Loyalty (not one of the specified emotions, but often associated with a sense of duty or commitment)
Oczekiwane: love
Tekst: i am just feeling cranky and blue
Odpowiedź modelu: Sadness
Oczekiwane: anger
Tekst: i can have for a treat or if i am feeling festive
Odpowiedź modelu: Joy
Oczekiwane: joy


Podsumowując: Wynik ogólnie jest dość niski dla Mistrala i widać, że ma problemy ze zrozumieniem polecenia dokładnie. Ale problem jest też w sumie z danymi, nie dziwie się, że model nie radzi sobie bez żadnego treningu, bo zdania są na tyle specyficzne, że sam miałbym problem je sklasyfikować.